In [21]:
import os
import glob
import pandas as pd

df1 = pd.read_csv("combined_2017.csv")
df2 = pd.read_csv("combined_2018.csv")
df3 = pd.read_csv("combined_2019.csv")
data = pd.concat([df1, df2,df3], ignore_index=True)
print(data.shape)

(22572043, 19)


In [22]:
data.drop(['ehail_fee'], axis=1)
data.drop(['store_and_fwd_flag'],axis = 1)
data = data[data['tolls_amount'] >= 0]
# base fare for green taxi
data = data[data['fare_amount']>=2.5]
data = data[data['passenger_count'] <= 6]
data = data[data['passenger_count'] > 0 ]
data = data[data['RatecodeID'] <= 6]
data = data[data['trip_distance'] > 0]

In [23]:
data['lpep_pickup_datetime'] = pd.to_datetime(data['lpep_pickup_datetime'] )
data['lpep_dropoff_datetime'] = pd.to_datetime(data['lpep_dropoff_datetime'] )
data['pickup_year'] = pd.DatetimeIndex(data['lpep_pickup_datetime']).year
data['pickup_month'] = pd.DatetimeIndex(data['lpep_pickup_datetime']).month
data['pickup_day'] = pd.DatetimeIndex(data['lpep_pickup_datetime']).day
data['pickup_hour'] = pd.DatetimeIndex(data['lpep_pickup_datetime']).hour
data['pickup_minutes'] = pd.DatetimeIndex(data['lpep_pickup_datetime']).minute
data['dropOff_year'] = pd.DatetimeIndex(data['lpep_dropoff_datetime']).year
data['dropOff_month'] = pd.DatetimeIndex(data['lpep_dropoff_datetime']).month
data['dropOff_day'] = pd.DatetimeIndex(data['lpep_dropoff_datetime']).day
data['dropOff_hour'] = pd.DatetimeIndex(data['lpep_dropoff_datetime']).hour
data['dropOff_minutes'] = pd.DatetimeIndex(data['lpep_dropoff_datetime']).minute
data=data.drop(['ehail_fee'], axis=1)
data=data.drop(['store_and_fwd_flag'], axis=1)    

In [24]:
import datetime
trip_dur=[]

#adding column duration
for pu_yr, do_yr, pu_mon, do_mon, pu_day, do_day,pu_hr, do_hr, pu_min, do_min in zip(data.pickup_year, data.dropOff_year, data.pickup_month, data.dropOff_month, data.pickup_day, data.dropOff_day, data.pickup_hour, data.dropOff_hour, data.pickup_minutes, data.dropOff_minutes):
    pickup = datetime.datetime(pu_yr, pu_mon, pu_day,pu_hr, pu_min,0)
    dropoff = datetime.datetime(do_yr, do_mon, do_day,do_hr, do_min,0)
    trip_dur.append((dropoff - pickup).seconds/60)
data['duration']=trip_dur

In [25]:
#removing outliers
data = data[data['trip_distance'] > 0.5]
data = data[data['trip_distance'] < 500]
data = data[data['duration'] <  480]
data = data[data['duration'] >  1]

In [26]:
#adding speed
data['speed']=data['trip_distance']*60/data['duration']

In [27]:
data=data[data['speed'] < 50] 

In [28]:
print(len(data))
print(max(data['speed']), min(data['speed']))
print(max(data['trip_distance']), min(data['trip_distance']))
print(max(data['duration']), min(data['duration']))

20953187
49.98461538461538 0.06554621848739496
220.08 0.51
479.0 2.0


In [29]:
data.to_csv("171819_DATA.csv")